In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [22]:
df = pd.read_json('/home/taha/Downloads/data_samples/human_like_behavior_detection.json')

In [23]:
df

,_index,_id,_score,_source,_ignored
0,human_like_behavior_recognition,Sd6YSJMBHmkLybI7VQyc,1,"{'is_bot': False, 'user': {'_id': 'FdxYP5MBHmk...",NaN
1,human_like_behavior_recognition,Et6YSJMBHmkLybI7uSHX,1,"{'is_bot': False, 'user': {'_id': 'EttVP5MBHmk...",NaN
2,human_like_behavior_recognition,vN2ISJMBHmkLybI7A-KC,1,"{'is_bot': True, 'user': {'_id': '6TqpzZIB8zdc...",[tweets._source.legacy.full_text.keyword]
3,human_like_behavior_recognition,id6YSJMBHmkLybI7exTa,1,"{'is_bot': False, 'user': {'_id': 'T9xWP5MBHmk...",NaN
4,human_like_behavior_recognition,0d2ISJMBHmkLybI7CuRz,1,"{'is_bot': True, 'user': {'_id': 'CjqpzZIB8zdc...",NaN
...,...,...,...,...,...
9995,human_like_behavior_recognition,Pd6YSJMBHmkLybI7TQux,1,"{'is_bot': False, 'user': {'_id': 'INxYP5MBHmk...",NaN
9996,human_like_behavior_recognition,zt6YSJMBHmkLybI7kheV,1,"{'is_bot': False, 'user': {'_id': '59xXP5MBHmk...",NaN
9997,human_like_behavior_recognition,WN6YSJMBHmkLybI7khmV,1,"{'is_bot': False, 'user': {'_id': 'VdtVP5MBHmk...",[tweets._source.legacy.full_text.keyword]
9998,human_like_behavior_recognition,i96YSJMBHmkLybI7hRap,1,"{'is_bot': False, 'user': {'_id': '8dxWP5MBHmk...",NaN


In [24]:
df['is_bot'] = df['_source'].apply(lambda x: x['is_bot'])
df['label'] = df['is_bot'].apply(lambda x: 1 if x else 0)

In [25]:
df.drop(columns=['is_bot'], inplace=True)
print(df[['_source', 'label']].head())

                                             _source  label
0  {'is_bot': False, 'user': {'_id': 'FdxYP5MBHmk...      0
1  {'is_bot': False, 'user': {'_id': 'EttVP5MBHmk...      0
2  {'is_bot': True, 'user': {'_id': '6TqpzZIB8zdc...      1
3  {'is_bot': False, 'user': {'_id': 'T9xWP5MBHmk...      0
4  {'is_bot': True, 'user': {'_id': 'CjqpzZIB8zdc...      1


In [26]:
source_df = pd.json_normalize(df['_source'])

In [27]:
df_expanded = pd.concat([df.drop(columns=['_source']), source_df], axis=1)

In [28]:
df_expanded.head(10)

,_index,_id,_score,_ignored,label,is_bot,tweets,user._id,user._source.friends_count,user._source.media_count,...,user._source.name,user._source.meta_data.description.urls,user._source.location,user._source.lang,user._source.device,user._source.social_media,user._source.username,user._source.posts_count,user._source.updated_image_last_7day,user._source.updated_banner_last_7day
0,human_like_behavior_recognition,Sd6YSJMBHmkLybI7VQyc,1,NaN,0,False,"[{'_index': 'iran_tweets_v2', '_id': '17847404...",FdxYP5MBHmkLybI7qUPT,582,3,...,un heureux hasard,[],Tehran,fa,Twitter for iPhone,twitter,serendipitisme,50,0.0,0.0
1,human_like_behavior_recognition,Et6YSJMBHmkLybI7uSHX,1,NaN,0,False,[],EttVP5MBHmkLybI7Budf,974,1,...,Selena,[],,None,Twitter Web App,twitter,DavidDominican1,162,NaN,NaN
2,human_like_behavior_recognition,vN2ISJMBHmkLybI7A-KC,1,[tweets._source.legacy.full_text.keyword],1,True,"[{'_index': 'iran_tweets_v2', '_id': '17857939...",6TqpzZIB8zdcL10qX9wX,17,0,...,Javad Hosseini,[],ایران,fa,Twitter for Android,twitter,JavadHoseini66,5,0.0,0.0
3,human_like_behavior_recognition,id6YSJMBHmkLybI7exTa,1,NaN,0,False,"[{'_index': 'iran_tweets_v2', '_id': '17840232...",T9xWP5MBHmkLybI7aQjX,255,27,...,Ghulam Murtaza Saich,[],کھیوہ پنجاب پاکستان,fa,Twitter for Android,twitter,GhulamM71508442,848,0.0,0.0
4,human_like_behavior_recognition,0d2ISJMBHmkLybI7CuRz,1,NaN,1,True,[],CjqpzZIB8zdcL10qqePs,135,0,...,Saam,[],,None,Twitter for iPhone,twitter,saamkasheero,349,NaN,NaN
5,human_like_behavior_recognition,Ct2ISJMBHmkLybI7CuRz,1,NaN,1,True,"[{'_index': 'iran_tweets_v2', '_id': '17793187...",uDqpzZIB8zdcL10qjuBV,59,0,...,Myn,[],,fa,Twitter for Android,twitter,Myn11206191,92,0.0,0.0
6,human_like_behavior_recognition,Q96YSJMBHmkLybI7aBBw,1,NaN,0,False,"[{'_index': 'iran_tweets_v2', '_id': '17848484...",o9xXP5MBHmkLybI7gCO6,175,0,...,gustavo rock,[],,fa,Twitter for iPhone,twitter,gustavo82909210,1493,0.0,0.0
7,human_like_behavior_recognition,ht6YSJMBHmkLybI7cBP9,1,NaN,0,False,"[{'_index': 'iran_tweets_v2', '_id': '17689554...",hNxWP5MBHmkLybI7OgMO,365,109,...,amoory,[],urmia,fa,Twitter for Android,twitter,itsmeamoory,523,0.0,0.0
8,human_like_behavior_recognition,Wd6YSJMBHmkLybI7nRvN,1,NaN,0,False,"[{'_index': 'iran_tweets_v2', '_id': '17847727...",1NtVP5MBHmkLybI70vne,249,0,...,Kyle Reese,[],,fa,Twitter for Android,twitter,KyleReese344982,148,0.0,0.0
9,human_like_behavior_recognition,hN6YSJMBHmkLybI7cBL9,1,NaN,0,False,[],UdxXP5MBHmkLybI75i66,974,1,...,Selena,[],,None,Twitter Web App,twitter,DavidDominican1,162,NaN,NaN


In [29]:
for column in df_expanded.columns:
    print(f"Frequency of values in {column}:")
    print(df_expanded[column].value_counts())
    print('-' * 40)

Frequency of values in _index:
_index
human_like_behavior_recognition    10000
Name: count, dtype: int64
----------------------------------------
Frequency of values in _id:
_id
Sd6YSJMBHmkLybI7VQyc    1
IN2ISJMBHmkLybI7A-GB    1
0d6YSJMBHmkLybI7VQ2e    1
JN6YSJMBHmkLybI7VQyc    1
wd6YSJMBHmkLybI7Xw9j    1
                       ..
qN2ISJMBHmkLybI7A-KC    1
R92ISJMBHmkLybI7CuRz    1
Qd6YSJMBHmkLybI7phyv    1
Ft2ISJMBHmkLybI7IurE    1
k96YSJMBHmkLybI7aBFx    1
Name: count, Length: 10000, dtype: int64
----------------------------------------
Frequency of values in _score:
_score
1    10000
Name: count, dtype: int64
----------------------------------------
Frequency of values in _ignored:
_ignored
[tweets._source.legacy.full_text.keyword]                                                                       596
[tweets._source.legacy.retweeted_status_result.result.card.legacy.binding_values.value.string_value.keyword]    282
[tweets._source.legacy.retweeted_status_result.result.tweet.lega

In [30]:
df_expanded.drop(columns=['is_bot', '_index', '_id', '_score', 'user._source.is_verified', 'user._source.activity', 'user._source.predicted_gender', 'user._source.country_name', 
                          'user._source.profile_image', 'user._source.political_location', 'user._source.bio_sentence', 'user._source.meta_data.description.urls',
                          'user._source.lang', 'user._source.social_media', 'tweets', 'user._source.username', '_ignored'], inplace=True)

In [31]:
categorical = df_expanded.select_dtypes(include=['object']).columns
categorical

Index(['user._id', 'user._source.created_at', 'user._source.bio',
       'user._source.predicted_category', 'user._source.province_name',
       'user._source.predicted_lang', 'user._source.user_id',
       'user._source.name', 'user._source.location', 'user._source.device'],
      dtype='object')

In [32]:
label_encoder = LabelEncoder()
for column in categorical:
    df_expanded[column] = label_encoder.fit_transform(df_expanded[column])

In [33]:
df_expanded

,label,user._id,user._source.friends_count,user._source.media_count,user._source.favourites_count,user._source.created_at,user._source.is_blue_verified,user._source.bio,user._source.predicted_age,user._source.province_id,...,user._source.province_name,user._source.predicted_lang,user._source.user_id,user._source.followers_count,user._source.name,user._source.location,user._source.device,user._source.posts_count,user._source.updated_image_last_7day,user._source.updated_banner_last_7day
0,0,2547,582,3,9894,29,False,18,27,24,...,8,1,3,102,144,24,2,50,0.0,0.0
1,0,2406,974,1,199,73,False,16,-1,0,...,0,3,47,60,94,0,0,162,NaN,NaN
2,1,1100,17,0,1322,133,False,77,30,0,...,0,1,107,3,51,38,1,5,0.0,0.0
3,0,4700,255,27,1085,45,False,89,-1,27,...,6,3,19,110,39,45,1,848,0.0,0.0
4,1,2074,135,0,26,123,False,0,12,0,...,0,3,97,9,89,0,2,349,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,3002,1720,0,1577,122,False,36,-1,0,...,0,3,96,29,27,0,1,1529,NaN,NaN
9996,0,901,146,1,1086,63,False,24,-1,0,...,0,1,37,88,83,0,1,226,0.0,0.0
9997,0,5104,200,47,3585,76,False,64,-1,24,...,8,1,50,387,180,32,1,610,0.0,0.0
9998,0,1456,199,1,2,182,False,42,36,0,...,0,2,158,38,126,0,1,107,0.0,0.0


In [34]:
booleans = df_expanded.select_dtypes(include=['bool']).columns
df_expanded[booleans] = df_expanded[booleans].astype(int)

In [35]:
df_expanded

,label,user._id,user._source.friends_count,user._source.media_count,user._source.favourites_count,user._source.created_at,user._source.is_blue_verified,user._source.bio,user._source.predicted_age,user._source.province_id,...,user._source.province_name,user._source.predicted_lang,user._source.user_id,user._source.followers_count,user._source.name,user._source.location,user._source.device,user._source.posts_count,user._source.updated_image_last_7day,user._source.updated_banner_last_7day
0,0,2547,582,3,9894,29,0,18,27,24,...,8,1,3,102,144,24,2,50,0.0,0.0
1,0,2406,974,1,199,73,0,16,-1,0,...,0,3,47,60,94,0,0,162,NaN,NaN
2,1,1100,17,0,1322,133,0,77,30,0,...,0,1,107,3,51,38,1,5,0.0,0.0
3,0,4700,255,27,1085,45,0,89,-1,27,...,6,3,19,110,39,45,1,848,0.0,0.0
4,1,2074,135,0,26,123,0,0,12,0,...,0,3,97,9,89,0,2,349,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,3002,1720,0,1577,122,0,36,-1,0,...,0,3,96,29,27,0,1,1529,NaN,NaN
9996,0,901,146,1,1086,63,0,24,-1,0,...,0,1,37,88,83,0,1,226,0.0,0.0
9997,0,5104,200,47,3585,76,0,64,-1,24,...,8,1,50,387,180,32,1,610,0.0,0.0
9998,0,1456,199,1,2,182,0,42,36,0,...,0,2,158,38,126,0,1,107,0.0,0.0


In [36]:
nan_counts = df_expanded.isna().sum()
nan_counts

label                                       0
user._id                                    0
user._source.friends_count                  0
user._source.media_count                    0
user._source.favourites_count               0
user._source.created_at                     0
user._source.is_blue_verified               0
user._source.bio                            0
user._source.predicted_age                  0
user._source.province_id                    0
user._source.predicted_category             0
user._source.country_id                     0
user._source.province_name                  0
user._source.predicted_lang                 0
user._source.user_id                        0
user._source.followers_count                0
user._source.name                           0
user._source.location                       0
user._source.device                         0
user._source.posts_count                    0
user._source.updated_image_last_7day     1977
user._source.updated_banner_last_7

In [37]:
df_cleaned = df_expanded.dropna()
df_cleaned

,label,user._id,user._source.friends_count,user._source.media_count,user._source.favourites_count,user._source.created_at,user._source.is_blue_verified,user._source.bio,user._source.predicted_age,user._source.province_id,...,user._source.province_name,user._source.predicted_lang,user._source.user_id,user._source.followers_count,user._source.name,user._source.location,user._source.device,user._source.posts_count,user._source.updated_image_last_7day,user._source.updated_banner_last_7day
0,0,2547,582,3,9894,29,0,18,27,24,...,8,1,3,102,144,24,2,50,0.0,0.0
2,1,1100,17,0,1322,133,0,77,30,0,...,0,1,107,3,51,38,1,5,0.0,0.0
3,0,4700,255,27,1085,45,0,89,-1,27,...,6,3,19,110,39,45,1,848,0.0,0.0
5,1,9065,59,0,91,51,0,0,-1,0,...,0,1,25,5,72,0,1,92,0.0,0.0
6,0,8152,175,0,4931,57,0,0,-1,0,...,0,2,31,138,119,0,2,1493,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,2098,199,1,2,182,0,42,36,0,...,0,2,158,38,126,0,1,107,0.0,0.0
9996,0,901,146,1,1086,63,0,24,-1,0,...,0,1,37,88,83,0,1,226,0.0,0.0
9997,0,5104,200,47,3585,76,0,64,-1,24,...,8,1,50,387,180,32,1,610,0.0,0.0
9998,0,1456,199,1,2,182,0,42,36,0,...,0,2,158,38,126,0,1,107,0.0,0.0


In [38]:
len(df_cleaned)

8023

In [39]:
df_train = df_cleaned[:6017]
df_val = df_cleaned[6018:7040]
df_test = df_cleaned[7041:]

In [40]:
X_train = df_train.drop(columns=['label'])
y_train = df_train['label']

X_val = df_val.drop(columns=['label'])
y_val = df_val['label']

X_test = df_test.drop(columns=['label'])
y_test = df_test['label']

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(input_size, 64) 
        self.layer2 = nn.Linear(64, 32)  
        self.output = nn.Linear(32, 1) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.layer1(x))  
        x = torch.relu(self.layer2(x))  
        x = self.sigmoid(self.output(x)) 
        return x

model = MLP(X_train.shape[1])

criterion = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
batch_size = 32

for epoch in range(num_epochs):
    model.train()

    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), y_train)  

    optimizer.zero_grad()
    loss.backward()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs.squeeze(), y_val)

model.eval()  
with torch.no_grad():
    test_outputs = model(X_test)
    test_predictions = (test_outputs.squeeze() > 0.5).float() 

test_accuracy = accuracy_score(y_test.numpy(), test_predictions.numpy())
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Test Accuracy: 79.43%
